# Matching Theoretical to Observed

In [1]:
import geopandas as gpd
import folium
import json
from tqdm import tqdm
import re

from custom_functions.gtfs_methods import detect_direction, match_schedule_for_service_line
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)

## Single Observation

In [2]:
ht = pd.read_pickle("data/computed/schedule_normalized_dist.pkl")
obs = pd.read_pickle("data/computed/observed_time_ES.pkl")

In [3]:
ht_g = ht.groupby(by=["route_short_name", "trip_id","service_id", "date_normalized"])
obs_g = obs.groupby(by=["date", "stop_name", "stop_id_cleaned", "stop_name__terminus"])

In [63]:
test_schedule = ht_g.get_group(("5", 112949577236270000, 236270000, "15-09-2021"))
test_schedule

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,time_seconds,headway_min,clusters,cluster_agg_value,regularity,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,start_date_ft,end_date_ft,label,date_normalized,stop_id_clean_c,descr_fr,descr_nl,geometry,geom_position_before,dist_m
2204418,112949577236270000,23:42:28,23:42:28,8261,1,0,0,NaN,HERRMANN-DEBROUX,NaN,50.812195,4.428503,NaN,NaN,0,28.0,1,236270000,ERASME,1,8912850,005m0123,5,ERASME - HERRMANN-DEBROUX,NaN,1,NaN,E6B012,FFFFFF,85348,15.0,cluster_3,10.0,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays,15-09-2021,8261,HERRMANN-DEBROUX,HERRMANN-DEBROUX,POINT (154211.1 166863),None,NaN
2204419,112949577236270000,23:43:25,23:43:25,8251,2,0,0,NaN,DEMEY,NaN,50.813545,4.421005,NaN,NaN,0,27.0,1,236270000,ERASME,1,8912850,005m0123,5,ERASME - HERRMANN-DEBROUX,NaN,1,NaN,E6B012,FFFFFF,85405,15.0,cluster_5,15.0,0,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays,15-09-2021,8251,DEMEY,DEMEY,POINT (153682.6 167012.8),POINT (154211.1 166863),549.319843
2204420,112949577236270000,23:44:52,23:44:52,8241,3,0,0,NaN,BEAULIEU,NaN,50.815128,4.409347,NaN,NaN,0,26.0,1,236270000,ERASME,1,8912850,005m0123,5,ERASME - HERRMANN-DEBROUX,NaN,1,NaN,E6B012,FFFFFF,85492,15.0,cluster_5,15.0,0,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays,15-09-2021,8241,BEAULIEU,BEAULIEU,POINT (152860.9 167188.3),POINT (153682.6 167012.8),840.232789
2204421,112949577236270000,23:46:15,23:46:15,8231,4,0,0,NaN,DELTA,NaN,50.818685,4.404406,NaN,NaN,0,25.0,1,236270000,ERASME,1,8912850,005m0123,5,ERASME - HERRMANN-DEBROUX,NaN,1,NaN,E6B012,FFFFFF,85575,15.0,cluster_4,10.0,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays,15-09-2021,8231,DELTA,DELTA,POINT (152512.5 167583.8),POINT (152860.9 167188.3),527.070024
2204422,112949577236270000,23:47:10,23:47:10,8221,5,0,0,NaN,HANKAR,NaN,50.821592,4.405618,NaN,NaN,0,24.0,1,236270000,ERASME,1,8912850,005m0123,5,ERASME - HERRMANN-DEBROUX,NaN,1,NaN,E6B012,FFFFFF,85630,15.0,cluster_4,10.0,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays,15-09-2021,8221,HANKAR,HANKAR,POINT (152597.7 167907.3),POINT (152512.5 167583.8),334.531448
2204423,112949577236270000,23:48:31,23:48:31,8211,6,0,0,NaN,PETILLON,NaN,50.827929,4.403859,NaN,NaN,0,23.0,1,236270000,ERASME,1,8912850,005m0123,5,ERASME - HERRMANN-DEBROUX,NaN,1,NaN,E6B012,FFFFFF,85711,15.0,cluster_4,10.0,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays,15-09-2021,8211,PETILLON,PETILLON,POINT (152473.4 168612.1),POINT (152597.7 167907.3),715.676973
2204424,112949577236270000,23:49:47,23:49:47,8201,7,0,0,NaN,THIEFFRY,NaN,50.833690,4.401677,NaN,NaN,0,22.0,1,236270000,ERASME,1,8912850,005m0123,5,ERASME - HERRMANN-DEBROUX,NaN,1,NaN,E6B012,FFFFFF,85787,15.0,cluster_4,10.0,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays,15-09-2021,8201,THIEFFRY,THIEFFRY,POINT (152319.4 169252.9),POINT (152473.4 168612.1),659.045249
2204425,112949577236270000,23:51:19,23:51:19,8071,8,0,0,NaN,MERODE,NaN,50.839534,4.397696,NaN,NaN,0,21.0,1,236270000,ERASME,1,8912850,005m0123,5,ERASME - HERRMANN-DEBROUX,NaN,1,NaN,E6B012,FFFFFF,85879,15.0,cluster_4,10.0,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays,15-09-2021,8071,MERODE,MERODE,POINT (152038.7 169902.9),POINT (152319.4 169252.9),708.020120
2204426,112949577236270000,23:53:10,23:53:10,8061,9,0,0,NaN,SCHUMAN,NaN,50.842972,4.380974,NaN,NaN,0,20.0,1,236270000,ERASME,1,8912850,005m0123,5,ERASME - HERRMANN-DEBROUX,NaN,1,NaN,E6B012,FFFFFF,85990,15.0,cluster_4,10.0,1,1,1,1,1,1,0,0,20210901,20210917,2021-09-01,2021-09-17,workdays,15-09-2021,8061,SCHUMAN,SCHUMAN,POINT (150860.8 170285),POINT (152038.7 16

In [79]:
def turn_sec_to_hours(time_in_sec: float) -> str:
    hours = int(time_in_sec / 3600)
    minutes = int(((time_in_sec / 3600) - hours) * 60)
    seconds = int(time_in_sec - hours * 3600 - minutes * 60)
    if len(str(minutes)) == 1:
        minutes = "0" + str(minutes)
    if len(str(hours)) == 1:
        hours = "0" + str(hours)
    if len(str(seconds)) == 1:
        seconds = "0" + str(seconds)
    return str(hours) + ":" + str(minutes) + ":" + str(seconds)
    
def match_schedule_for_service_line_optim(
    schedule: pd.DataFrame,
    observations: pd.DataFrame,
    vehicle_id: int,
    date: str,
    midnight_tresh: int = 7200,
) -> pd.DataFrame:
    speeds_ms = {
        3: 4.333333333333333,  # Bus
        0: 4.527777777777778,  # Tram
        1: 7.749999999999999,  # Metro
    }
    d = []
    times_ser = 0
    for row in schedule.sort_values("time_seconds").to_dict("records"):
        stop_id_cleaned = re.sub("\D", "", row["stop_id"].lstrip("0"))

        if row["time_seconds"] > 86400:
            time_to_match_mid = row["time_seconds"] - 86400
            dts = date.split("-")
            dts[0] = str(int(dts[0]) + 1)
            dts = "-".join(dts)
            es_mid = observations.get_group(
                (dts, row["stop_name"], stop_id_cleaned, row["trip_headsign"])
            )
            del dts

            if es_mid.empty == False:
                es_mid["diff_time"] = abs(
                    es_mid["adjusted_arrival_time(ts)"] - time_to_match_mid
                )

        es = observations.get_group(
            (date, row["stop_name"], stop_id_cleaned, row["trip_headsign"])
        )

        if es.empty:
            rec_matched = {}
            rec_matched["vehicule_id"] = vehicle_id
            d.append(temp)

        else:
            es["diff_time"] = abs(es["adjusted_arrival_time(ts)"] - row["time_seconds"])

            if row["time_seconds"] > 86400:
                es = pd.concat([es, es_mid])
                del es_mid

            rec_matched = es[es.diff_time == min(es.diff_time)].to_dict("records")[0]

            if (
                (
                    (rec_matched["adjusted_arrival_time(ts)"] < midnight_tresh) &
                    (rec_matched["adjusted_arrival_time(ts)"] + 86400 < times_ser)
                    )
                | ((rec_matched["adjusted_arrival_time(ts)"] > midnight_tresh) &
                    (rec_matched["adjusted_arrival_time(ts)"] < times_ser))
            ) & (rec_matched["diff_time"] < row["cluster_agg_value"] * 60):

                delta_time = row["dist_m"] / speeds_ms[row["route_type"]]

                rec_matched["adjusted_arrival_time(ts)"] = times_ser + delta_time
                rec_matched["adjusted_arrival_time"] = turn_sec_to_hours(
                    rec_matched["adjusted_arrival_time(ts)"]
                )
                rec_matched["approx_rec"] = 1

            if rec_matched["diff_time"] > row["cluster_agg_value"] * 60:
                rec_matched = {}
            else:
                times_ser = rec_matched["adjusted_arrival_time(ts)"]

            rec_matched["vehicule_id"] = vehicle_id

            del es
            temp = {**row, **rec_matched} if rec_matched != {} else row
            d.append(temp)

    vehicle_id += 1
    del times_ser
    return d

In [81]:
test = match_schedule_for_service_line_optim(
    schedule= test_schedule,
    observations= obs_g,
    vehicle_id = 0,
    date= "11-09-2021"
)
pd.DataFrame(test).to_csv("data/CSVs/hard_case2.csv")

## Whole data set

In [ ]:
ht_group = ht.groupby(by=[
    "route_short_name",
    "trip_id",
    "service_id"
])

obs_to_collect = []
ID_v = 0

for name, group in tqdm(ht_group):
    r_name = name[0]
    vehicle_ = name[1]
    service_ = name[2]

    # Theoretical schedule to search
    sched = ht[(ht.route_short_name == r_name) & (ht.service_id == service_) & (ht.trip_id == vehicle_)]

    # obs schedule to search
    obs_sample = obs[obs.route_short_name == r_name]

    try:
        # Get direction
        direction = detect_direction(
            ligne = r_name, 
            geom = geom, 
            schedule= sched
        )

        # running the match
        match = match_schedule_for_service_line_test(
            schedule = sched,
            exceptions = exceptions,
            observations = obs_sample,
            geometry_table = direction,
            vehicle_id = ID_v
        )
        obs_to_collect.append(match[0])
        ID_v = match[1]
        del match, direction, obs_sample
        
    except Exception as inst:
        print(inst)
        print(name)
        break

In [ ]:
l = []
for name, group in ht_group:
    l.append(name)

In [ ]:
dfs = np.array_split(pd.DataFrame(l, columns=["route", "trip_id", "service_id"]), 10)

In [ ]:
for index, ldf in enumerate(dfs):
    ldf.to_pickle("data/chunks/chunk_" + str(index))

## Test Playground

In [ ]:
ht_group = ht.groupby(by=[
    "route_short_name",
    "trip_id",
    "service_id"
])

obs_to_collect = []
ID_v = 0

for name, group in tqdm(ht_group):
    r_name = name[0]
    vehicle_ = name[1]
    service_ = name[2]

    # Theoretical schedule to search
    sched = ht[(ht.route_short_name == r_name) & (ht.service_id == service_) & (ht.trip_id == vehicle_)]

    try :
        # obs schedule to search
        obs_sample = obs[obs.route_short_name == r_name]

        # Get direction
        direction = detect_direction(
            ligne = r_name, 
            geom = geom, 
            schedule= sched
        )

        # running the match
        match = match_schedule_for_service_line(
            schedule = sched,
            exceptions = exceptions,
            observations = obs_sample,
            geometry_table = direction,
            vehicle_id = ID_v
        )
        obs_to_collect.append(match[0])
        ID_v = match[1]
        del match, direction, obs_sample
    except Exception as inst:
        print(inst)
        print(name)
        break

In [ ]:
def turn_sec_to_hours(time_in_sec: float) -> str:
    hours = int(time_in_sec / 3600)
    minutes = int(((time_in_sec / 3600) - hours) * 60)
    seconds = int(time_in_sec - hours * 3600 - minutes * 60)
    if len(str(minutes)) == 1:
        minutes = "0" + str(minutes)
    if len(str(hours)) == 1:
        hours = "0" + str(hours)
    if len(str(seconds)) == 1:
        seconds = "0" + str(seconds)
    return str(hours) + ":" + str(minutes) + ":" + str(seconds)


def extract_schedule(
    schedule: pd.DataFrame,
    transportation_type: int,
    thresh_gaps: int = 120,
    distance_col: str = "distance_from_point",
    speeds_ms: dict[int, float] = {
        3: 4.333333333333333,  # Bus
        0: 4.527777777777778,  # Tram
        1: 7.749999999999999,  # Metro
    },
) -> pd.DataFrame:
    if schedule.shape[0] > 1:

        schedule["lag_-1"] = schedule[distance_col].shift(1)
        schedule["lag_+1"] = schedule[distance_col].shift(-1)
        schedule["t_lag_before"] = schedule.sort_values("time_seconds")[
            "time_seconds"
        ].shift(1)
        schedule["t_diff(sec)"] = schedule["time_seconds"] - schedule["t_lag_before"]

        arr = schedule[(schedule["lag_-1"] > schedule[distance_col])]

        if (schedule.iloc[0][distance_col] > 0) | (
            schedule.iloc[0][distance_col] == 0 & schedule.iloc[1][distance_col] > 0
        ):
            arr = pd.concat([arr, schedule.head(1)])

        dep = schedule[(schedule["lag_+1"] > schedule[distance_col])]
        gaps = schedule[schedule["t_diff(sec)"] > thresh_gaps]

        FS = pd.concat([arr, dep, gaps])

    else:
        FS = schedule

    FS["adjusted_arrival_time(ts)"] = round(
        FS.time_seconds - (FS.distance_from_point / speeds_ms.get(transportation_type)),
        3,
    )
    FS["adjusted_arrival_time"] = FS["adjusted_arrival_time(ts)"].apply(
        turn_sec_to_hours
    )

    return FS.sort_values(by="adjusted_arrival_time(ts)")


def define_date_window(
    schedule_start_date: pd._libs.tslibs.timestamps.Timestamp,
    schedule_end_date: pd._libs.tslibs.timestamps.Timestamp,
    label: str,
    service_id: int,
    exceptions: pd.DataFrame,
    obs_start_date: str = "2021-09-06 00:00:00",
    obs_end_date: str = "2021-09-21 00:00:00",
) -> list[str]:
    labels_dict = {
        "workdays": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"],
        "saturday": ["Saturday"],
        "sunday": ["Sunday"],
    }

    min_date = pd.to_datetime(obs_start_date)
    max_date = pd.to_datetime(obs_end_date)

    added_dates = pd.to_datetime(
        exceptions[
            (exceptions.service_id == service_id) & (exceptions.exception_type == 1)
        ].date_ft
    )
    removed_dates = pd.to_datetime(
        exceptions[
            (exceptions.service_id == service_id) & (exceptions.exception_type == 2)
        ].date_ft
    )

    service_window = pd.date_range(start=schedule_start_date, end=schedule_end_date)
    data_window = pd.date_range(start=min_date, end=max_date)

    woi_raw = [
        date
        for date in data_window
        if (date in service_window) & (date.strftime("%A") in labels_dict[label])
    ]

    woi_raw.extend(
        [dt for dt in added_dates if (dt not in woi_raw) & (dt not in data_window)]
    )

    woi = [dt.strftime("%d-%m-%Y") for dt in woi_raw if dt not in removed_dates]

    return woi


def search_time(
    schedule: pd.DataFrame,
    expected_time: float,
    headway: float,
    stop_dist: float,
    speed_vehicle: float,
    time_last_val: float,
) -> dict:
    schedule["diff_time"] = abs(schedule["adjusted_arrival_time(ts)"] - expected_time)
    rec_matched = schedule[schedule.diff_time == min(schedule.diff_time)].to_dict(
        "records"
    )[0]

    t_diff = rec_matched["diff_time"]
    t_time = rec_matched["adjusted_arrival_time(ts)"]

    if (t_time < time_last_val) & (t_diff < headway * 60):
        delta_time = stop_dist / speed_vehicle
        rec_matched["adjusted_arrival_time(ts)"] = time_last_val + delta_time
        rec_matched["adjusted_arrival_time"] = turn_sec_to_hours(
            rec_matched["adjusted_arrival_time(ts)"]
        )
        rec_matched["approx_rec"] = 1

    return rec_matched


def detect_direction(
    ligne: str, geom: pd.DataFrame, schedule: pd.DataFrame
) -> pd.DataFrame:
    g = geom[(geom.ligne_cleaned == ligne)]
    g_dir = pd.DataFrame()

    for name, group in g.groupby("VARIANTE"):
        sched_stops = schedule.sort_values("time_seconds").stop_name.to_list()

        hset_fr = group.sort_values("succession").descr_fr.to_list()
        hset_nl = group.sort_values("succession").descr_nl.to_list()

        for i in range(len(hset_fr) - len(sched_stops) + 1):
            if (sched_stops == hset_fr[i : i + len(sched_stops)]) | (
                sched_stops == hset_nl[i : i + len(sched_stops)]
            ):
                g_dir = group

    return g_dir
    
def match_schedule_for_service_line_test(
    schedule: pd.DataFrame,
    exceptions: pd.DataFrame,
    observations: pd.DataFrame,
    geometry_table: pd.DataFrame,
    vehicle_id: int,
) -> pd.DataFrame:
    speeds_ms = {
        3: 4.333333333333333,  # Bus
        0: 4.527777777777778,  # Tram
        1: 7.749999999999999,  # Metro
    }

    d = []

    sch = schedule.iloc[0]
    start_date = sch.start_date_ft
    end_date = sch.end_date_ft
    lab = sch.label
    service_id = sch.service_id
    doi = define_date_window(start_date, end_date, lab, service_id, exceptions)

    for date in doi:
        times_ser = [0]
        for row in schedule.sort_values("time_seconds").to_dict("records"):
            stop_name = row["stop_name"]
            stop_id = re.sub("\D", "", row["stop_id"].lstrip("0"))
            direction = row["trip_headsign"]
            transport_type = row["route_type"]
            expected_time = row["time_seconds"]
            headway = row["cluster_agg_value"]
            stop_dist_table = geometry_table[geometry_table.descr_fr == stop_name]
            if stop_dist_table.empty:
                stop_dist_table = geometry_table[geometry_table.descr_nl == stop_name]
            stop_dist = stop_dist_table.iloc[0].dist_m

            es = extract_schedule(
                observations[
                    (observations.stop_name == stop_name)
                    & (observations.stop_id_cleaned == stop_id)
                    & (observations.date == date)
                    & (observations.stop_name__terminus == direction)
                ],
                transport_type,
            )

            if not es.empty:
                rec_matched = search_time(
                    es,
                    expected_time=expected_time,
                    headway=headway,
                    stop_dist=stop_dist,
                    speed_vehicle=speeds_ms[transport_type],
                    time_last_val=times_ser[-1],
                )

                if rec_matched["diff_time"] > headway * 60:
                    rec_matched = {}
                else:
                    times_ser.append(rec_matched["adjusted_arrival_time(ts)"])

                rec_matched["dist_previous_stop"] = stop_dist
                rec_matched["date_searched"] = date
                rec_matched["vehicule_id"] = vehicle_id

                temp = {**row, **rec_matched} if rec_matched != {} else row
                d.append(temp)
        vehicle_id += 1

    return (pd.DataFrame(d), vehicle_id)

In [ ]:
obs_1 = obs_sample = obs[obs.route_short_name == "1"]
sched = ht[(ht.route_short_name == "1") & (ht.service_id == 236269500) & (ht.trip_id == 112947203236269500)]

direction = detect_direction(
    ligne = r_name, 
    geom = geom, 
    schedule= sched
    )

In [ ]:
match = match_schedule_for_service_line_test(
    schedule = sched,
    exceptions = exceptions,
    observations = obs_1,
    geometry_table = direction,
    vehicle_id = 0
)

In [6]:
obs = pd.read_pickle("data/computed/observed_time.pkl")
obs.route_short_name.unique().shape

(74,)